In [2]:
import warnings

import pandas as pd

warnings.filterwarnings('ignore')

In [16]:

change_dtype = {5: str, 32: "category", 34: "category", 55: str, 62: str, 63: str, 64: str, 77: "category",
                80: "category", 91: "category", 93: "category", 95: "category", 97: str, 115: str, 116: "category",
                122: "category"}

df: pd.DataFrame = pd.read_csv("../data/raw/globalterrorismdb_0221dist.csv", index_col=0, dtype=change_dtype)

In [17]:
df.shape

(201183, 135)

In [5]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
eventid,201183.0,NaN,NaN,NaN,200422456548.324463,1342944201.424622,197000000001.0,199202240012.5,201103080007.0,201507170012.5,201912310033.0
iyear,201183.0,NaN,NaN,NaN,2004.158478,13.429481,1970.0,1992.0,2011.0,2015.0,2019.0
imonth,201183.0,NaN,NaN,NaN,6.451678,3.387376,0.0,4.0,6.0,9.0,12.0
iday,201183.0,NaN,NaN,NaN,15.520983,8.808477,0.0,8.0,15.0,23.0,31.0
approxdate,11754,2946,"September 18-24, 2016",101,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
INT_LOG,201183.0,NaN,NaN,NaN,-4.45893,4.543925,-9.0,-9.0,0.0,0.0,1.0
INT_IDEO,201183.0,NaN,NaN,NaN,-4.374699,4.64054,-9.0,-9.0,-9.0,0.0,1.0
INT_MISC,201183.0,NaN,NaN,NaN,0.087746,0.545811,-9.0,0.0,0.0,0.0,1.0
INT_ANY,201183.0,NaN,NaN,NaN,-3.878002,4.682635,-9.0,-9.0,0.0,0.0,1.0


In [18]:
# filtrowanie kolumn i wyrzucenie tych w których większość wierszy jest pusta
df = df[(df.crit1 == 1) & (df.crit2 == 1) & (df.crit3 == 1) & (df.doubtterr == 0)]
df.dropna(axis=1, thresh=int(0.5 * df.shape[0]), inplace=True)

In [9]:
df.columns

Index(['eventid', 'iyear', 'imonth', 'iday', 'extended', 'country',
       'country_txt', 'region', 'region_txt', 'provstate', 'city', 'latitude',
       'longitude', 'specificity', 'vicinity', 'summary', 'crit1', 'crit2',
       'crit3', 'doubtterr', 'multiple', 'success', 'suicide', 'attacktype1',
       'attacktype1_txt', 'targtype1', 'targtype1_txt', 'targsubtype1',
       'targsubtype1_txt', 'corp1', 'target1', 'natlty1', 'natlty1_txt',
       'gname', 'guncertain1', 'individual', 'nperps', 'nperpcap', 'claimed',
       'weaptype1', 'weaptype1_txt', 'weapsubtype1', 'weapsubtype1_txt',
       'weapdetail', 'nkill', 'nkillus', 'nkillter', 'nwound', 'nwoundus',
       'nwoundte', 'property', 'ishostkid', 'scite1', 'dbsource', 'INT_LOG',
       'INT_IDEO', 'INT_MISC', 'INT_ANY'],
      dtype='object')

In [19]:
# Usunięcie i zamiana niespójnie wprowadzonych danych
df.nperps = df.nperps.apply(lambda x: -99 if x == -9 else x)
df.nperpcap = df.nperpcap.apply(lambda x: -99 if x == -9 else x)

# usuniecie wartości pustych z kolumn docelowych
df.dropna(subset=["nkill", "nwound"], inplace=True)
# wypełnienie wartosci pustych zerami
df["nkillter"].fillna(0, inplace=True)
df["nwoundte"].fillna(0, inplace=True)
# usunięcie z ogólnej liczby zgonów/rannych terrorystów
df["nkill"] = df["nkill"] - df["nkillter"]
df["nwound"] = df["nwound"] - df["nwoundte"]
# Utworzenie kolumn z sumaryczną liczba zgonów i ofiar oraz klasy zerojedynkowej.
df["ncasualties"] = df["nkill"] + df["nwound"]
df["cas_class"] = df["ncasualties"].apply(lambda x: 0 if x == 0 else 1)

In [21]:
df.shape

(141051, 60)

In [22]:
df.to_csv("../data/interm/globalterrordb.csv")